# IPL Data Analysis

## Match wins by teams

In [2]:
SELECT T.name AS Team
, COUNT(M.match_id) AS Wins
FROM dimteam T
LEFT JOIN dimmatch M ON T.team_id = M.winner_team_id
GROUP BY T.name
ORDER BY Wins DESC;

## Championships by team

In [5]:
SELECT T.name AS Team
, COUNT(M.match_id) AS Wins
FROM dimteam T
INNER JOIN dimmatch M ON T.team_id = M.winner_team_id AND M.stage = 'Final'
GROUP BY T.name
ORDER BY Wins DESC;

## Matches Played, Wins and Championships

In [9]:
WITH MatchesPlayed AS (
    SELECT DISTINCT team_id, match_id 
    FROM dimteam T
    INNER JOIN factinnings F ON T.team_id = F.bat_team_id
    UNION
    SELECT DISTINCT team_id, match_id 
    FROM dimteam T
    INNER JOIN factinnings F ON T.team_id = F.bat_team_id
)
SELECT T.name AS Team
, COUNT(M1.match_id) AS MatchesPlayed
, COUNT(M.match_id) AS Wins
, ROUND((CAST(COUNT(M.match_id) AS FLOAT)/COUNT(M1.match_id))*100, 2) AS [Win%]
, COUNT(C.match_id) AS Championships
FROM dimteam T
INNER JOIN MatchesPlayed M1 ON M1.team_id = T.team_id
LEFT JOIN dimmatch M ON T.team_id = M.winner_team_id AND M.match_id = M1.match_id
LEFT JOIN dimmatch C ON T.team_id = C.winner_team_id AND C.match_id = M1.match_id AND C.stage = 'Final'
GROUP BY T.name
ORDER BY Wins DESC;

In [2]:
SELECT MAX(date) FROM dimmatch;

In [10]:
SELECT * FROM dimmatch WHERE date = '2025-05-08';

## Balls taken to hit a Six by Player (Min 50 sixes)

In [19]:
SELECT P.player_name
, SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END) AS BallsFaced
, SUM(CASE WHEN bat_runs = 6 THEN 1 ELSE 0 END) AS Sixes
, (SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END)/SUM(CASE WHEN bat_runs = 6 THEN 1 END)) AS BallsPerSix 
FROM factinnings F
INNER JOIN dimplayer P ON F.batter_id = P.player_id
GROUP BY P.player_name
HAVING SUM(CASE WHEN bat_runs = 6 THEN 1 ELSE 0 END) >= 50
ORDER BY BallsPerSix ASC, Sixes DESC;

## Balls taken for boundary (Min 50 boundaries)

In [5]:
SELECT P.player_name
, SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END) AS BallsFaced
, SUM(CASE WHEN bat_runs >= 4 THEN 1 ELSE 0 END) AS Boundary
, (SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END)/SUM(CASE WHEN bat_runs >= 4 THEN 1 END)) AS BallsPerBoundary
FROM factinnings F
INNER JOIN dimplayer P ON F.batter_id = P.player_id
GROUP BY P.player_name
HAVING SUM(CASE WHEN bat_runs >= 4 THEN 1 ELSE 0 END) >= 50
ORDER BY BallsPerBoundary ASC, Boundary DESC;

## Balls taken for boundary in powerplay (Min 50 boundaries)

In [11]:
SELECT P.player_name
, SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END) AS BallsFaced
, SUM(CASE WHEN bat_runs >= 4 THEN 1 ELSE 0 END) AS Boundary
, (SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END)/SUM(CASE WHEN bat_runs >= 4 THEN 1 END)) AS BallsPerBoundary
FROM factinnings F
INNER JOIN dimplayer P ON F.batter_id = P.player_id
WHERE F.[over] BETWEEN 0 AND 5
GROUP BY P.player_name
HAVING SUM(CASE WHEN bat_runs >= 4 THEN 1 ELSE 0 END) >= 50
ORDER BY BallsPerBoundary ASC, Boundary DESC;

## Ipl 2025 Top 10 Run Getters

In [6]:
SELECT TOP 10 P.player_name
, SUM(I.bat_runs) AS Runs
FROM factinnings I
INNER JOIN dimplayer P ON I.batter_id = P.player_id
INNER JOIN dimmatch M ON M.match_id = I.match_id
WHERE YEAR(M.[date]) = 2025
GROUP BY P.player_name
ORDER BY Runs DESC;

In [4]:
SELECT 1, 'Powerplay' 
, P.player_name
, SUM(F.bat_runs) AS Runs
, SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END) AS BallsFaced
, ROUND(CAST(SUM(F.bat_runs) AS FLOAT)*100/SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END), 2)AS SR
, SUM(CASE WHEN bat_runs >= 4 THEN 1 ELSE 0 END) AS Boundary
, (SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END)/SUM(CASE WHEN bat_runs >= 4 THEN 1 END)) AS BallsPerBoundary
FROM factinnings F
INNER JOIN dimplayer P ON F.batter_id = P.player_id
WHERE F.[over] BETWEEN 0 AND 5
--AND P.player_name IN ('TM Head', 'Abhishek Sharma', 'Ishan Kishan')
GROUP BY P.player_name
HAVING SUM(F.bat_runs) >= 500
UNION
SELECT 2, 'Middle Overs' 
, P.player_name
, SUM(F.bat_runs) AS Runs
, SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END) AS BallsFaced
, ROUND(CAST(SUM(F.bat_runs) AS FLOAT)*100/SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END), 2)AS SR
, SUM(CASE WHEN bat_runs >= 4 THEN 1 ELSE 0 END) AS Boundary
, (SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END)/SUM(CASE WHEN bat_runs >= 4 THEN 1 END)) AS BallsPerBoundary
FROM factinnings F
INNER JOIN dimplayer P ON F.batter_id = P.player_id
WHERE F.[over] BETWEEN 6 AND 14
--AND P.player_name IN ('TM Head', 'Abhishek Sharma', 'Ishan Kishan')
GROUP BY P.player_name
HAVING SUM(F.bat_runs) >= 500
UNION
SELECT 3, 'Death' 
, P.player_name
, SUM(F.bat_runs) AS Runs
, SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END) AS BallsFaced
, ROUND(CAST(SUM(F.bat_runs) AS FLOAT)*100/SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END), 2)AS SR
, SUM(CASE WHEN bat_runs >= 4 THEN 1 ELSE 0 END) AS Boundary
, (SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END)/SUM(CASE WHEN bat_runs >= 4 THEN 1 END)) AS BallsPerBoundary
FROM factinnings F
INNER JOIN dimplayer P ON F.batter_id = P.player_id
WHERE F.[over] BETWEEN 15 AND 19
--AND P.player_name IN ('TM Head', 'Abhishek Sharma', 'Ishan Kishan')
GROUP BY P.player_name
HAVING SUM(F.bat_runs) >= 500
ORDER BY 1, BallsPerBoundary ASC, Boundary DESC;

In [4]:
SELECT *
FROM dimplayer
WHERE player_name LIKE '%Kishan%'